<a href="https://colab.research.google.com/github/moonisali/ADFB_SpringerNature/blob/master/Task1_Word_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1: Word Embeddings (10 points)

This notebook will guide you through all steps necessary to train a word2vec model (Detailed description in the PDF).

## Imports

This code block is reserved for your imports. 

You are free to use the following packages: 

(List of packages)

In [58]:
# Imports

import pandas as pd
import numpy as np
import itertools as it
import spacy
from spacy.lang.hi import Hindi
nlp_hi = Hindi()
import regex as re
!pip install demoji
import demoji
demoji.download_codes()
!pip install emot
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
!pip install emoji

     |████████████████████████████████| 133kB 6.6MB/s 


# 1.1 Get the data (0.5 points)

The Hindi portion HASOC corpus from [github.io](https://hasocfire.github.io/hasoc/2019/dataset.html) is already available in the repo, at data/hindi_hatespeech.tsv . Load it into a data structure of your choice. Then, split off a small part of the corpus as a development set (~100 data points).

If you are using Colab the first two lines will let you upload folders or files from your local file system.

In [2]:
#TODO: implement!

#from google.colab import files
#uploaded = files.upload()


data_set = pd.read_csv("hindi_hatespeech.tsv",sep='\t')

#Making dev set:   
##we use sample function to randomly pick 100 rows as specified in question.
dev_set = data_set.sample(n=100)
dev_set.head()

,text_id,text,task_1,task_2,task_3
2859,hasoc_hi_1498,सुअर बुखारी अब तेरे कौम का अंत निकट आते जा रहा...,HOF,HATE,TIN
3853,hasoc_hi_2940,अब यह बताना भी गुनाह है की #कुत्ते भी योगा कर ...,HOF,OFFN,TIN
3324,hasoc_hi_7576,नेता चालाक है वो कुर्सी के लिए धर्म के नाम पर ...,NOT,NONE,NONE
265,hasoc_hi_3051,कार्यकर्ताओं द्वारा असंभव को संभव बनाने की महा...,NOT,NONE,NONE
3504,hasoc_hi_4231,शर्मिंदगी निरुत्तर कर ही देती है : स्वच्छ इ...,NOT,NONE,NONE


## 1.2 Data preparation (0.5 + 0.5 points)

* Prepare the data by removing everything that does not contain information. 
User names (starting with '@') and punctuation symbols clearly do not convey information, but we also want to get rid of so-called [stopwords](https://en.wikipedia.org/wiki/Stop_word), i. e. words that have little to no semantic content (and, but, yes, the...). Hindi stopwords can be found [here](https://github.com/stopwords-iso/stopwords-hi/blob/master/stopwords-hi.txt) Then, standardize the spelling by lowercasing all words.
Do this for the development section of the corpus for now.

* What about hashtags (starting with '#') and emojis? Should they be removed too? Justify your answer in the report, and explain how you accounted for this in your implementation.

In [ ]:
import pandas as pd
import numpy as np
import itertools as it
import spacy
from spacy.lang.hi import Hindi
import regex as re
import emoji
#from spacymoji import Emoji
nlp_hi = Hindi()
demoji.download_codes()



stop_words = open("stop_word_hindi.txt","r")
stopword_list = stop_words.readlines()
stop_words.close()
converted_list = []
for element in stopword_list:
    converted_list.append(element.strip())
print(converted_list)

for stopword in stopword_list:
    lexeme = nlp_hi.vocab[stopword]
    lexeme.is_stop = True


In [94]:
def preprocessing_hi(tweet):
  tweet_hi = []
  
  tweet = re.sub('http\S+\s*', '', tweet)  # remove URLs
  tweet = re.sub('RT|cc', '', tweet)  # remove RT and cc
  tweet = re.sub('#\S+', '', tweet)  # remove hashtags
  tweet = re.sub('@\S+', '', tweet)  # remove mentions
  tweet = demoji.replace(tweet," ") #remove smoji's
  tweet = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', tweet)  # remove punctuations
  tweet = re.sub('\s+', ' ', tweet)  # remove extra whitespace
    
    
  # ******* the following piece shall check for hindi specific text-preprocessing using spacy.hindi. lan. library ***** 
  # ******* as well as catches exception from previous block **********************************************************  
  
  tokenized_text = nlp_hi(tweet)
  for token in tokenized_text:
    if(token.text!='\n\n' 
      and (re.search('#\S+',token.text)is None)
      and (re.search(r'@\S+',token.text) is None)
      and not token.is_stop 
      and not token.is_punct 
      and not token.is_space 
      and not token.like_email
      and not token.is_digit
      and not token.is_quote
      and not token.like_url):
     tweet_hi.append(token.lemma_)
  
  
  tweet = ' '.join([token  for token in tweet_hi])
   
  return tweet

In [98]:
#from bs4 import BeautifulSoup

tes = " #gol #gol So 9 आज दिल्ली के तालकटोरा स्टेडियम में शिक्षा विभाग,दिल्ली सरकार के सौजन्य से आयोजित 'हैपीनेस एजुकेशन' के समापन समारोह में सम्मिलित हुआ 😀। इस कार्यक्रम में दिल्ली सरकार के विधायक,शिक्षा विभाग के उच्चाधिकारी व स्कूली बच्चों ने अपनी उपस्थिति दर्ज कराई। #Budget2019  @AAPDelhi @AamAadmiParty  https://t.co/zhNmdReb3A"
print(tes)
print("Cleaned Tweet is:",clean_tweet(tes))
now = clean_tweet(tes) 
tweet = print("Pre-processed tweet is:", preprocessing_hi(now))

#**** All the tweets from dev.set are pre-processed ****
cleaned_tweets = []
for t in dev_set.text:
  cleaned_tweets.append(preprocessing_hi(t))

df = pd.DataFrame(cleaned_tweets, columns = ['cleaned_tweets'])
print(cleaned_tweets[:-5])

 #gol #gol So 9 आज दिल्ली के तालकटोरा स्टेडियम में शिक्षा विभाग,दिल्ली सरकार के सौजन्य से आयोजित 'हैपीनेस एजुकेशन' के समापन समारोह में सम्मिलित हुआ 😀। इस कार्यक्रम में दिल्ली सरकार के विधायक,शिक्षा विभाग के उच्चाधिकारी व स्कूली बच्चों ने अपनी उपस्थिति दर्ज कराई। #Budget2019  @AAPDelhi @AamAadmiParty  https://t.co/zhNmdReb3A
Cleaned Tweet is:  So 9 आज दिल्ली के तालकटोरा स्टेडियम में शिक्षा विभाग दिल्ली सरकार के सौजन्य से आयोजित हैपीनेस एजुकेशन के समापन समारोह में सम्मिलित हुआ । इस कार्यक्रम में दिल्ली सरकार के विधायक शिक्षा विभाग के उच्चाधिकारी व स्कूली बच्चों ने अपनी उपस्थिति दर्ज कराई। 
Pre-processed tweet is: So आज दिल्ली तालकटोरा स्टेडियम शिक्षा विभाग दिल्ली सरकार सौजन्य आयोजित हैपीनेस एजुकेशन समापन समारोह सम्मिलित कार्यक्रम दिल्ली सरकार विधायक शिक्षा विभाग उच्चाधिकारी स्कूली बच्चों उपस्थिति दर्ज कराई
['सुअर बुखारी अब तेरे कौम अंत निकट आते इतना समझ ले', 'अब बताना गुनाह योगा क्यों हद्द भाई दिमाग प्रभु', 'नेता चालाक वो कुर्सी धर्म नाम लड़वाते जनता मुर्ख धर्म नाम आपस लड़ती', 'कार्यकर्त

## 1.3 Build the vocabulary (0.5 + 0.5 points)

The input to the first layer of word2vec is an one-hot encoding of the current word. The output od the model is then compared to a numeric class label of the words within the size of the skip-gram window. Now

* Compile a list of all words in the development section of your corpus and save it in a variable ```V```.

In [ ]:
#TODO: implement!
V = 

* Then, write a function ```word_to_one_hot``` that returns a one-hot encoding of an arbitrary word in the vocabulary. The size of the one-hot encoding should be ```len(v)```.

In [ ]:
#TODO: implement!
def word_to_one_hot(word):
  pass

## 1.4 Subsampling (0.5 points)

The probability to keep a word in a context is given by:

$P_{keep}(w_i) = \Big(\sqrt{\frac{z(w_i)}{0.001}}+1\Big) \cdot \frac{0.001}{z(w_i)}$

Where $z(w_i)$ is the relative frequency of the word $w_i$ in the corpus. Now,
* Calculate word frequencies
* Define a function ```sampling_prob``` that takes a word (string) as input and returns the probabiliy to **keep** the word in a context.

In [ ]:
#TODO: implement!
def sampling_prob(word):
  pass

# 1.5 Skip-Grams (1 point)

Now that you have the vocabulary and one-hot encodings at hand, you can start to do the actual work. The skip gram model requires training data of the shape ```(current_word, context)```, with ```context``` being the words before and/or after ```current_word``` within ```window_size```. 

* Have closer look on the original paper. If you feel to understand how skip-gram works, implement a function ```get_target_context``` that takes a sentence as input and [yield](https://docs.python.org/3.9/reference/simple_stmts.html#the-yield-statement)s a ```(current_word, context)```.

* Use your ```sampling_prob``` function to drop words from contexts as you sample them. 

In [ ]:
#TODO: implement!

def get_target_context(sentence):
  pass

# 1.6 Hyperparameters (0.5 points)

According to the word2vec paper, what would be a good choice for the following hyperparameters? 

* Embedding dimension
* Window size

Initialize them in a dictionary or as independent variables in the code block below. 

In [ ]:
# Set hyperparameters
window_size = 
embedding_size = 

# More hyperparameters
learning_rate = 0.05
epochs = 100

# 1.7 Pytorch Module (0.5 + 0.5 + 0.5 points)

Pytorch provides a wrapper for your fancy and super-complex models: [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). The code block below contains a skeleton for such a wrapper. Now,

* Initialize the two weight matrices of word2vec as fields of the class.

* Override the ```forward``` method of this class. It should take a one-hot encoding as input, perform the matrix multiplications, and finally apply a log softmax on the output layer.

* Initialize the model and save its weights in a variable. The Pytorch documentation will tell you how to do that.

In [ ]:
# Create model 

class Word2Vec(Module):
  def __init__(self):
    super().__init__()


  def forward(self, one_hot):
    pass

Word2Vec(
  (input): Linear(in_features=534, out_features=300, bias=False)
  (output): Linear(in_features=300, out_features=534, bias=False)
)


# 1.8 Loss function and optimizer (0.5 points)

Initialize variables with [optimizer](https://pytorch.org/docs/stable/optim.html#module-torch.optim) and loss function. You can take what is used in the word2vec paper, but you can use alternative optimizers/loss functions if you explain your choice in the report.

In [ ]:
# Define optimizer and loss
optimizer = 
criterion = 

# 1.9 Training the model (3 points)

As everything is prepared, implement a training loop that performs several passes of the data set through the model. You are free to do this as you please, but your code should:

* Load the weights saved in 1.6 at the start of every execution of the code block
* Print the accumulated loss at least after every epoch (the accumulate loss should be reset after every epoch)
* Define a criterion for the training procedure to terminate if a certain loss value is reached. You can find the threshold by observing the loss for the development set.

You can play around with the number of epochs and the learning rate.

In [ ]:
# Define train procedure

# load initial weights

def train():
 
  print("Training started")

train()

print("Training finished")

# 1.10 Train on the full dataset (0.5 points)

Now, go back to 1.1 and remove the restriction on the number of sentences in your corpus. Then, reexecute code blocks 1.2, 1.3 and 1.6 (or those relevant if you created additional ones). 

* Then, retrain your model on the complete dataset.

* Now, the input weights of the model contain the desired word embeddings! Save them together with the corresponding vocabulary items (Pytorch provides a nice [functionality](https://pytorch.org/tutorials/beginner/saving_loading_models.html) for this).